**Tabla de contenido**

- [Introducción](#Introduccion)
- [Lectura de datos](#Lectura-de-datos)
- [Preparación del dataframe](#Preparacion-del-dataframe)
- [Análisis exploratorio de los datos](#Analisis-exploratorio-de-los-datos)
    - [Categorías más frecuentadas](#Categorias-mas-frecuentadas)

# Introduccion

En este cuaderno examinaremos la información disponible sobre las interacciones generadas por los videos con mayor tendencia en YouTube en USA. El objetivo es identificar las categorías dominantes, los atributos más relevantes y los factores clave que contribuyen a que un video se vuelva tendencia, así como analizar si la fecha de publicación influye significativamente en su popularidad.

Además, se entrenarán modelos predictivos para estimar la cantidad de “likes” o visitas que podría alcanzar un video. No siendo más, comensemos.

# Lectura de datos

In [1]:
import pandas as pd
import numpy as np
import re
import json
import os
file_path = lambda file: os.path.join(os.getcwd(),'data',file)


Empecemos leyendo el archivo `Json` que contiene las categorias de los videos. Para ello, lo haremos de la siguiente forma:

- `pd.json_normalize`: Esta función de pandas toma datos en formato `JSON`, incluso si son complejos y anidados, y los transforma en un DataFrame plano y estructurado, lo que facilita su manipulación y análisis. Sus parámetros principales son:
    - `data`: Es el objeto `Json` que contiene los datos a procesar.
    - `record_path`: Es la lista de interes. Cada elemento dentro de la lista, representa una fila en el dataframe.
    - `meta`: Con esto se indica la lista de columnas (claves) que quieres incluir y que existendo dentro de la lista de interes.

In [2]:
data = json.load(open('data/US_category_id.json'))
df_Category = pd.json_normalize(
    data=data,
    record_path='items',
)

df_Category = df_Category[["id", "snippet.channelId", "snippet.title", "snippet.assignable"]]
df_Category.columns = ["id", "channelId", "title", "assignable"]
df_Category.head()

,id,channelId,title,assignable
0,1,UCBR8-60-B28hp2BmDPdntcQ,Film & Animation,True
1,2,UCBR8-60-B28hp2BmDPdntcQ,Autos & Vehicles,True
2,10,UCBR8-60-B28hp2BmDPdntcQ,Music,True
3,15,UCBR8-60-B28hp2BmDPdntcQ,Pets & Animals,True
4,17,UCBR8-60-B28hp2BmDPdntcQ,Sports,True


Perfecto!. Ahora leamos los datos del archivo csv.

In [3]:
df_vid= pd.read_csv(file_path('USvideos.csv'))
df_vid.head(2)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."


# Preparacion del dataframe

Dado que contamos con ambos DataFrames (df_vid y df_Category) correctamente leídos, es momento de organizar y ajustar los tipos de datos que contienen. En esta sección realizaremos lo siguiente:

- `Corrección del formato de fecha`: Convertiremos las fechas al tipo de dato datetime.
- `Ajuste de tipos de datos`: Verificaremos que cada columna posea el tipo de dato adecuado (number u object, según corresponda).
- `Merge`.

In [4]:
df_Category.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          32 non-null     object
 1   channelId   32 non-null     object
 2   title       32 non-null     object
 3   assignable  32 non-null     bool  
dtypes: bool(1), object(3)
memory usage: 932.0+ bytes


In [5]:
df_Category['id']= df_Category['id'].astype('Int64')

Perfecto!. Cada columna contiene su respectivo tipo de dato. Veamos ahora el dataframe de los videos.

In [6]:
df_vid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40949 entries, 0 to 40948
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   video_id                40949 non-null  object
 1   trending_date           40949 non-null  object
 2   title                   40949 non-null  object
 3   channel_title           40949 non-null  object
 4   category_id             40949 non-null  int64 
 5   publish_time            40949 non-null  object
 6   tags                    40949 non-null  object
 7   views                   40949 non-null  int64 
 8   likes                   40949 non-null  int64 
 9   dislikes                40949 non-null  int64 
 10  comment_count           40949 non-null  int64 
 11  thumbnail_link          40949 non-null  object
 12  comments_disabled       40949 non-null  bool  
 13  ratings_disabled        40949 non-null  bool  
 14  video_error_or_removed  40949 non-null  bool  
 15  de

Aquí solo tenemos dos variables a las cuales se les necesita definir correctamente el formato, estas son:
- trending_date: Día en que el el video empezó a estar en tendencia.
- publish_time: Día en el que el video fue públicado.

In [7]:
df_vid['trending_date'] = pd.to_datetime(df_vid['trending_date'], format='%y.%d.%m')
df_vid['publish_time'] = pd.to_datetime(df_vid['publish_time'],utc=True)
df_vid['publish_time'] = df_vid['publish_time'].dt.tz_convert('America/New_York')
df_vid.head(2)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,2017-11-14,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13 12:13:01-05:00,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,2017-11-14,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13 02:30:00-05:00,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."


Para finalizar esta sección, asignaremos la categoría correspondiente a cada video. Para ello, realizaremos un merge entre los dos DataFrames (df_vid y df_Category), utilizando como columnas clave category_id (en df_vid) e id (en df_Category).

In [8]:
df_vid = pd.merge(df_vid,df_Category,left_on='category_id',right_on='id', how='left')
df_vid.head(1)

,video_id,trending_date,title_x,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,id,channelId,title_y,assignable
0,2kyS6SvSYSE,2017-11-14,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13 12:13:01-05:00,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,22,UCBR8-60-B28hp2BmDPdntcQ,People & Blogs,True


In [9]:
df_vid.rename(columns={'title_y': 'category'}, inplace=True)
df_vid = df_vid.drop(columns=['category_id','id'],axis=1)

Con esto ya estamos listos para explorar los datos.

# Analisis exploratorio de los datos

El propósito principal del análisis exploratorio de datos es comprender el estado y las características de la información antes de decidir qué técnicas de ciencia de datos o machine learning aplicar. Esto implica organizar los datos, conocer su contenido, identificar las variables más relevantes y analizar cómo se relacionan entre sí, detectar patrones, evaluar la presencia de datos faltantes y valores atípicos, y finalmente extraer conclusiones a partir de este proceso.

En síntesis, el análisis exploratorio es una etapa clave para entender, visualizar y extraer información valiosa del dataset, lo que nos permite elegir con fundamento la estrategia o técnica más adecuada para su procesamiento posterior.

Este análisis exploratorio siempre inicia con las preguntas de investigación y es el paso cero antes de aplicar ciencia de datos. En este caso las preguntas son las siguientes:

`EDA`

1. ¿Cuáles son las categorías de vídeos que reciben mayores vistas y Likes?
2. ¿Es posible encontrar agrupaciones o clasificaciones de videos?
3. ¿Cuál es la combinación de características o atributos más importantes que hacen de un video tendencia?
4. ¿La temporada o fecha en el que el video es publicado tiene alguna influencia?
5. ¿Qué canales publican con mayor frecuencia y cuál es su promedio de likes y views?
6. ¿Cómo se distribuye la duración en tendencias para cada categoría?
7. ¿Existe relación entre la cantidad de comentarios y los likes recibidos?
8. ¿Qué etiquetas (tags) son más frecuentes y cómo influyen en views y likes?
9. ¿Cuál es la distribución temporal de las publicaciones (hora del día, día de la semana, mes)?
10. ¿Se observan picos de visualizaciones relacionados con eventos o fechas especiales (festivos, lanzamientos, etc.)?
11. ¿Cuántos videos fueron eliminados o presentaron errores y cómo afecta eso a las métricas?
12. ¿Cuál es el tiempo estimado desde la publicación hasta entrar en tendencias?

`Modelado`

1. ¿Es posible predecir la cantidad de likes que recibirá un video?
2. En caso afirmativo, ¿qué variables pueden utilizarse para realizar dichas predicciones?
3. ¿Se puede estimar la probabilidad de que un video entre en tendencias?

`Hipótesis`

- La fecha y la hora de publicación de un video influyen significativamente en la cantidad de likes que este recibirá.

Dicho esto, procedamos con la exploración de los datos.

In [10]:
print(f"la cantidad de resgistros que tiene el dataframe son: {df_vid.shape[0]}")
print(f"La cantidad de características son: {df_vid.shape[1]}")

la cantidad de resgistros que tiene el dataframe son: 40949
La cantidad de características son: 18


In [11]:
val_nan = {col:df_vid[col].isnull().sum() for col in df_vid.columns}
val_nan = pd.DataFrame.from_dict(val_nan, orient='index',columns=['nulos'])
val_nan.head(len(val_nan))

,nulos
video_id,0
trending_date,0
title_x,0
channel_title,0
publish_time,0
tags,0
views,0
likes,0
dislikes,0
comment_count,0


Podemos observar que el DataFrame cuenta con un total de `18 características` y `40949 registros`, de los cuales únicamente la columna `description presenta 570 registros sin descripción`.
Dado que me interesa conservar todo el conjunto de datos, asignaré la frase `"No description"` a aquellos registros que carecen de descripción, es decir, reemplazaré los valores NaN en esa columna por dicha frase. 

Además, crearé una nueva columna indicadora que tome el valor 1 para los videos con descripción y 0 para aquellos que no la tienen.

In [12]:
df_vid['con_descripcion'] = df_vid['description'].isnull().map({False:1,True:0})
df_vid['description'] = df_vid['description'].fillna('No description')

Ahora veamos si hay registros dúplicados.

In [13]:
total_duplicados = df_vid.duplicated().sum()
print(f"Duplicados (sin contar la primera aparición): {total_duplicados}")

Duplicados (sin contar la primera aparición): 48


In [14]:
df_vid = df_vid.drop_duplicates() # Elimina duplicados conservando la primera aparición

## Categorias mas frecuentadas

In [15]:
df_vid.head()

,video_id,trending_date,title_x,channel_title,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,channelId,category,assignable,con_descripcion
0,2kyS6SvSYSE,2017-11-14,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,2017-11-13 12:13:01-05:00,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,UCBR8-60-B28hp2BmDPdntcQ,People & Blogs,True,1
1,1ZAPwfrtAFY,2017-11-14,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,2017-11-13 02:30:00-05:00,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True,1
2,5qpjK5DgCt4,2017-11-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,2017-11-12 14:05:24-05:00,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,UCBR8-60-B28hp2BmDPdntcQ,Comedy,True,1
3,puqaWrEC7tY,2017-11-14,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,2017-11-13 06:00:04-05:00,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...,UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True,1
4,d380meD0W0M,2017-11-14,I Dare You: GOING BALD!?,nigahiga,2017-11-12 13:01:41-05:00,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True,1
